In [1]:
print("Building my first ANN project with tensorflow")

Building my first ANN project with tensorflow


In [2]:
import pandas as pd  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle 

In [16]:
## load the dataset 
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
## Preprocess the data 
# drop the columns that are not needed
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [18]:
df['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [19]:
## apply label encoding to gender and one hot encoding to geography 
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])

In [20]:
## apply one hot encoding to geography
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
encoded_geo = ohe.fit_transform(df[['Geography']]).toarray()
encoded_geo_df = pd.DataFrame(encoded_geo, columns=ohe.get_feature_names_out(['Geography']))

In [21]:
# concatentate the encoded columns with the original dataframe
df = pd.concat([df, encoded_geo_df], axis=1)
df.drop(['Geography'], axis=1, inplace=True)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [22]:
## save the encoders and scalers 
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(le, f)
with open('one_hot_encoder_geo.pkl', 'wb') as f:
    pickle.dump(ohe, f)

In [23]:
## divide the dataset into independent and dependent variables
X = df.drop(['Exited'], axis=1)
y = df['Exited']

In [24]:
## split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
## save the scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

**Train the aritificial neural network**

In [26]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [28]:
## build the ann model 
model = Sequential([
    Dense(64, activation = 'relu', input_shape = (X_train.shape[1],)), # 1st hidden layer connected with input layer 
    Dense(32, activation = 'relu'), # 2nd hidden layer connected with 1st hidden layer
    Dense(1, activation = 'sigmoid') # output layer connected with 2nd hidden layer
]) # instantiating the model


In [ ]:
model.summary() # print the summary of the model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001) # define the optimizer
loss = tf.keras.losses.BinaryCrossentropy() # define the loss function

In [31]:
# compile the model 
model.compile(optimizer = opt, loss = loss, metrics = ['accuracy']) # binary_crossentropy is used for binary classification problems

In [ ]:
## set up the tensorboard 
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1) # create a tensorboard callback

In [36]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True) # create an early stopping callback

In [37]:
## train the model 
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test),
    epochs = 100,
    callbacks = [tensorboard_callback, early_stopping_callback],
)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3204 - accuracy: 0.8662 - val_loss: 0.3454 - val_accuracy: 0.8600
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3185 - accuracy: 0.8674 - val_loss: 0.3469 - val_accuracy: 0.8620
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3164 - accuracy: 0.8702 - val_loss: 0.3459 - val_accuracy: 0.8615
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3158 - accuracy: 0.8681 - val_loss: 0.3443 - val_accuracy: 0.8655
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3131 - accuracy: 0.8699 - val_loss: 0.3455 - val_accuracy: 0.8630
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3128 - accuracy: 0.8700 - val_loss: 0.3465 - val_accuracy: 0.8655
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3094 - accuracy: 0.8716 - val_loss: 0.3460 - val_accuracy: 0.8645

In [38]:
model.save('model.h5')

c:\Users\Lenovo\OneDrive\Desktop\dl\ann\customer_churn_prediction\model\deep_learning\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [39]:
# load the tensorboard extension 
%load_ext tensorboard

In [41]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 5328), started 0:02:11 ago. (Use '!kill 5328' to kill it.)

**Prediction using trained ANN model**

In [42]:
import tensorflow as tf 
from tensorflow.keras.models import load_model
import pickle
import pandas as pd 
import numpy as np

In [43]:
## load the trained model, scaler, label encoder and one hot encoder
model = load_model('model.h5')
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('label_encoder_gender.pkl', 'rb') as f:
    le = pickle.load(f)
with open('one_hot_encoder_geo.pkl', 'rb') as f:
    ohe = pickle.load(f)

In [45]:
model.summary() # print the summary of the model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
# Example input data for prediction 
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender' : 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [51]:
def preprocess_input(input_data):
    df = pd.DataFrame([input_data])
    df['Gender'] = le.transform(df['Gender'])
    encoded_geo = ohe.transform(df[['Geography']]).toarray()
    df.drop('Geography', axis = 1, inplace = True)
    encoded_geo_df = pd.DataFrame(encoded_geo, columns = ohe.get_feature_names_out(['Geography']))
    df = pd.concat([df, encoded_geo_df], axis = 1)
    df = scaler.transform(df)
    return df



In [52]:
temp_df = preprocess_input(input_data)

In [54]:
temp_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [55]:
## prediction 
prediction = model.predict(temp_df)

1/1 [==============================] - 0s 230ms/step


In [56]:
print(prediction)

[[0.03544248]]


In [57]:
prediction_proba = prediction[0][0]

In [58]:
prediction_proba

0.03544248

**Testing / Experimenting**

In [1]:
import pandas as pd 
import numpy as np 
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [2]:
import tensorflow as tf 

model = tf.keras.models.load_model('model.h5')

In [3]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pickle

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('label_encoder_gender.pkl', 'rb') as f:
    le = pickle.load(f)
with open('one_hot_encoder_geo.pkl', 'rb') as f:
    ohe = pickle.load(f)

In [34]:
X = df.drop(['Exited'], axis=1)
y = df['Exited']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
X_test.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
6252,6253,15687492,Anderson,596,Germany,Male,32,3,96709.07,2,0,0,41788.37
4684,4685,15736963,Herring,623,France,Male,43,1,0.00,2,1,1,146379.30
1731,1732,15721730,Amechi,601,Spain,Female,44,4,0.00,2,1,0,58561.31
4742,4743,15762134,Liang,506,Germany,Male,59,8,119152.10,2,1,1,170679.74
4521,4522,15648898,Chuang,560,Spain,Female,27,7,124995.98,1,1,1,114669.79


In [37]:
def preprocess_input(input_data):
    # Create a copy of input data
    df = input_data.copy()
    
    # First drop unnecessary columns if they exist
    columns_to_drop = ['RowNumber', 'CustomerId', 'Surname']
    existing_cols = [col for col in columns_to_drop if col in df.columns]
    if existing_cols:
        df.drop(existing_cols, axis=1, inplace=True)
    
    # Label encode Gender
    df['Gender'] = le.transform(df['Gender'])
    
    # One-hot encode Geography
    encoded_geo = ohe.transform(df[['Geography']]).toarray()
    encoded_geo_df = pd.DataFrame(encoded_geo, columns=ohe.get_feature_names_out(['Geography']))
    
    # Drop Geography column and concatenate encoded features
    df.drop(['Geography'], axis=1, inplace=True)
    df = pd.concat([df, encoded_geo_df], axis=1)
    
    # Scale the features
    scaled_df = scaler.transform(df)
    
    return scaled_df

In [38]:
X_test_preprocessed = preprocess_input(X_test)

In [41]:
X_test.shape

(2000, 13)

In [42]:
X_test.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary'],
      dtype='object')

In [43]:
encoded_geo = ohe.transform(X_test[['Geography']]).toarray()
encoded_geo.shape 

(2000, 3)

In [45]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [46]:
encoded_geo_df = pd.DataFrame(encoded_geo, columns=ohe.get_feature_names_out(['Geography']))
encoded_geo_df.shape

(2000, 3)

In [47]:
encoded_geo_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,1.0,0.0
4,0.0,0.0,1.0


In [48]:
encoded_geo_df.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

In [49]:
X_test.shape

(2000, 13)

In [50]:
encoded_geo_df.shape

(2000, 3)

In [51]:
X_test.drop(['Geography', 'RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)
X_test.shape

(2000, 9)

In [52]:
X_test.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,596,Male,32,3,96709.07,2,0,0,41788.37
1,623,Male,43,1,0.00,2,1,1,146379.30
2,601,Female,44,4,0.00,2,1,0,58561.31
3,506,Male,59,8,119152.10,2,1,1,170679.74
4,560,Female,27,7,124995.98,1,1,1,114669.79


In [53]:
X_test['Gender'] = le.transform(X_test['Gender'])
X_test.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,596,1,32,3,96709.07,2,0,0,41788.37
1,623,1,43,1,0.00,2,1,1,146379.30
2,601,0,44,4,0.00,2,1,0,58561.31
3,506,1,59,8,119152.10,2,1,1,170679.74
4,560,0,27,7,124995.98,1,1,1,114669.79


In [54]:
input_data = pd.concat([X_test, encoded_geo_df], axis=1)
input_data.shape

(2000, 12)

In [55]:
input_data = scaler.transform(input_data)
input_data.shape

(2000, 12)

In [56]:
y_pred = model.predict(input_data)
y_pred.shape

63/63 [==============================] - 0s 2ms/step


(2000, 1)

In [57]:
y_pred

array([[0.02597934],
       [0.01562653],
       [0.08857149],
       ...,
       [0.808738  ],
       [0.08047   ],
       [0.36204484]], dtype=float32)

In [58]:
y_test

6252    0
4684    0
1731    0
4742    0
4521    0
       ..
6412    1
8285    0
7853    1
1095    1
6929    1
Name: Exited, Length: 2000, dtype: int64

In [59]:
y_pred = [0 if i[0] < 0.5 else 1 for i in y_pred]
y_pred = np.array(y_pred)

In [60]:
y_pred.shape

(2000,)

In [62]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred)
cm

array([[1542,   65],
       [ 204,  189]], dtype=int64)

In [63]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.74      0.48      0.58       393

    accuracy                           0.87      2000
   macro avg       0.81      0.72      0.75      2000
weighted avg       0.86      0.87      0.85      2000

